### RNA Sequence project

#### Kehang Li

In [1]:
%matplotlib inline
import os
import requests
import urllib 

import pandas as pd
import numpy as np
import statsmodels.api as sm
import pymc3 as pm
import scipy

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from sklearn import linear_model
from scipy import stats as sps
from sklearn.feature_selection import SelectFromModel
from IPython.display import display
from scipy.stats import gamma
from scipy.stats import invgamma
from scipy.stats import norm
from scipy.stats import poisson
from scipy.stats import halfnorm
import random
import math

### Question 1: Importing Data

#### a. Load the file ‘RNASeqData.xlsx’ and save it as a pandas dataframe called data_df

In [2]:
data_df = pd.read_excel('/Users/likehang/Desktop/RNASeqData.xlsx')

#### b. Filter the dataframe to only contain the 60 columns.

In [3]:
# 140 columns in total
# Drop the normalized results (the column names with string 'Norm')
data_df = data_df.loc[:, ~data_df.columns.str.startswith('Norm')]

# Drop the comparing analysis results (the column names with string 'vs')
data_df = data_df[data_df.columns.drop(list(data_df.filter(regex='vs')))]

# Drop the average value (the column names with string 'Mean')
data_df = data_df[data_df.columns.drop(list(data_df.filter(regex='Mean')))]

data_df.drop(['RowID', 'Description','EntrezID','Class'], axis=1, inplace=True)

In [4]:
# Filtered data with 61 columns in total and renamed it as raw_data
raw_data = data_df

# Rename the ID with gene_ids
raw_data.rename(columns = {'ID':'gene_ids'}, inplace = True)

In [5]:
raw_data.head()

,gene_ids,Raw_10_HC_Auto_066_237,Raw_11_Orkambi_006_Base,Raw_12_Orkambi_007_V2,Raw_13_HC_Auto_068_239,Raw_14_Orkambi_007_Base,Raw_15_Orkambi_009_V2,Raw_16_HC_Auto_072_243,Raw_17_Orkambi_009_Base,Raw_18_Orkambi_010_V2,...,Raw_Orkambi_021_Base,Raw_Orkambi_021_V2,Raw_Orkambi_022_Base,Raw_Orkambi_022_V2,Raw_Orkambi_024_Base,Raw_Orkambi_024_V2,Raw_Orkambi_025_Base,Raw_Orkambi_025_V2,Raw_Orkambi_026_Base,Raw_Orkambi_026_V2
0,A1BG,81,38,112,46,64,196,125,140,72,...,113,89,31,125,44,78,113,98,38,70
1,A1BG-AS1,49,31,84,26,38,86,80,76,38,...,40,36,26,87,23,72,74,55,64,53
2,A2M-AS1,74,108,62,47,41,31,105,45,63,...,16,23,40,104,370,199,81,30,228,33
3,AAAS,161,86,197,92,116,143,243,136,151,...,130,71,28,178,62,219,176,106,160,121
4,AACS,199,128,198,117,162,263,218,243,201,...,192,157,71,228,130,185,198,114,131,157


### Question 2: Normalization

#### a. Normalize data by relative abundance of g compared to all other genes

In [6]:
#Total sum per column except 'gene_ids': 
raw_data.loc['Sum_cols',:]= raw_data.iloc[:, 1:].sum(axis=0)

# Divide each row by the sum of each column
raw_data_1 = raw_data.drop('gene_ids', axis=1)
raw_data_1 = raw_data_1.div(raw_data_1.iloc[-1])

# Multiply all the values with 1000000
raw_data_1 = raw_data_1.multiply(1000000)

# Rename columns
raw_data_1.columns = raw_data_1.columns.str.replace("Raw", "Norm")

In [29]:
raw_data_1.head()

,Norm_10_HC_Auto_066_237,Norm_11_Orkambi_006_Base,Norm_12_Orkambi_007_V2,Norm_13_HC_Auto_068_239,Norm_14_Orkambi_007_Base,Norm_15_Orkambi_009_V2,Norm_16_HC_Auto_072_243,Norm_17_Orkambi_009_Base,Norm_18_Orkambi_010_V2,Norm_19_HC_Auto_074_245,...,Norm_Orkambi_021_Base,Norm_Orkambi_021_V2,Norm_Orkambi_022_Base,Norm_Orkambi_022_V2,Norm_Orkambi_024_Base,Norm_Orkambi_024_V2,Norm_Orkambi_025_Base,Norm_Orkambi_025_V2,Norm_Orkambi_026_Base,Norm_Orkambi_026_V2
0,3.573084,2.084737,4.012814,2.405698,2.554380,5.560526,3.970076,3.336330,2.125438,4.603163,...,4.207276,3.280020,2.938322,3.631128,2.578384,3.192450,4.068087,3.258494,3.096321,3.045395
1,2.161495,1.700706,3.009610,1.359742,1.516663,2.439823,2.540848,1.811150,1.121759,2.478626,...,1.489301,1.326750,2.464399,2.527265,1.347792,2.946877,2.664057,1.828747,5.214856,2.305799
2,3.264299,5.925042,2.221379,2.457996,1.636400,0.879471,3.334864,1.072392,1.859759,8.940759,...,0.595720,0.847646,3.791383,3.021098,21.681863,8.144840,2.916063,0.997498,18.577924,1.435686
3,7.102057,4.718089,7.058253,4.811396,4.629815,4.056914,7.717827,3.241006,4.457517,6.594917,...,4.840229,2.616645,2.653968,5.170726,3.633177,8.963416,6.336136,3.524494,13.037139,5.264183
4,8.778318,7.022272,7.094081,6.118841,6.465776,7.461318,6.923812,5.790915,5.933516,6.727700,...,7.148645,5.786102,6.729704,6.623177,7.617952,7.571836,7.128153,3.790493,10.674158,6.830386


In [92]:
# join two datasets as norm_cpm_df
norm_cpm_df = pd.concat([raw_data, raw_data_1],axis=1) 
norm_cpm_df = norm_cpm_df[norm_cpm_df.columns.drop(list(norm_cpm_df.filter(regex='Raw')))]

In [28]:
norm_cpm_df.head()

,gene_ids,Norm_10_HC_Auto_066_237,Norm_11_Orkambi_006_Base,Norm_12_Orkambi_007_V2,Norm_13_HC_Auto_068_239,Norm_14_Orkambi_007_Base,Norm_15_Orkambi_009_V2,Norm_16_HC_Auto_072_243,Norm_17_Orkambi_009_Base,Norm_18_Orkambi_010_V2,...,Norm_Orkambi_021_Base,Norm_Orkambi_021_V2,Norm_Orkambi_022_Base,Norm_Orkambi_022_V2,Norm_Orkambi_024_Base,Norm_Orkambi_024_V2,Norm_Orkambi_025_Base,Norm_Orkambi_025_V2,Norm_Orkambi_026_Base,Norm_Orkambi_026_V2
0,A1BG,3.573084,2.084737,4.012814,2.405698,2.554380,5.560526,3.970076,3.336330,2.125438,...,4.207276,3.280020,2.938322,3.631128,2.578384,3.192450,4.068087,3.258494,3.096321,3.045395
1,A1BG-AS1,2.161495,1.700706,3.009610,1.359742,1.516663,2.439823,2.540848,1.811150,1.121759,...,1.489301,1.326750,2.464399,2.527265,1.347792,2.946877,2.664057,1.828747,5.214856,2.305799
2,A2M-AS1,3.264299,5.925042,2.221379,2.457996,1.636400,0.879471,3.334864,1.072392,1.859759,...,0.595720,0.847646,3.791383,3.021098,21.681863,8.144840,2.916063,0.997498,18.577924,1.435686
3,AAAS,7.102057,4.718089,7.058253,4.811396,4.629815,4.056914,7.717827,3.241006,4.457517,...,4.840229,2.616645,2.653968,5.170726,3.633177,8.963416,6.336136,3.524494,13.037139,5.264183
4,AACS,8.778318,7.022272,7.094081,6.118841,6.465776,7.461318,6.923812,5.790915,5.933516,...,7.148645,5.786102,6.729704,6.623177,7.617952,7.571836,7.128153,3.790493,10.674158,6.830386


### Question 3: Top 10 genes

#### a. Filter norm_cpm_df to contain only listed genes

In [30]:
top10_ci_genes = ['LOC105372578', 'MCEMP1', 'MMP9', 'SOCS3', 'ANXA3', 'G0S2', 'IL1R2', 'PFKFB3', 'OSM', 'SEMA6B']

In [37]:
norm_cpm_df_1 = norm_cpm_df[norm_cpm_df['gene_ids'].isin(top10_ci_genes)]

In [38]:
norm_cpm_df_1

,gene_ids,Norm_10_HC_Auto_066_237,Norm_11_Orkambi_006_Base,Norm_12_Orkambi_007_V2,Norm_13_HC_Auto_068_239,Norm_14_Orkambi_007_Base,Norm_15_Orkambi_009_V2,Norm_16_HC_Auto_072_243,Norm_17_Orkambi_009_Base,Norm_18_Orkambi_010_V2,...,Norm_Orkambi_021_Base,Norm_Orkambi_021_V2,Norm_Orkambi_022_Base,Norm_Orkambi_022_V2,Norm_Orkambi_024_Base,Norm_Orkambi_024_V2,Norm_Orkambi_025_Base,Norm_Orkambi_025_V2,Norm_Orkambi_026_Base,Norm_Orkambi_026_V2
555,ANXA3,55.272527,71.045638,20.995615,83.205775,307.603285,100.117838,81.402431,545.561369,388.040111,...,194.279334,237.451308,275.254376,294.615163,142.690100,77.069010,189.184064,313.879457,24.526118,123.643037
4206,G0S2,1.235140,5.431288,2.113893,1.464338,7.224107,9.106780,2.032679,10.032819,10.686232,...,7.483738,6.375768,9.288887,13.885432,2.109587,2.169229,6.192133,6.051490,1.385196,5.264183
5313,IL1R2,55.228415,93.483990,49.515256,83.414966,154.100984,136.686807,83.847998,183.188321,183.643783,...,112.144376,153.755527,201.796338,143.531209,142.338503,88.651873,115.634484,182.209685,117.497218,108.981636
6886,LOC105372578,3.484860,7.570887,2.508009,4.759098,29.854322,11.745193,3.430145,77.140705,82.537858,...,16.047220,23.734075,88.623568,100.335316,17.111092,5.811896,13.176283,21.579213,2.688910,14.617896
8139,MCEMP1,5.866916,4.882673,2.758809,4.759098,30.053883,9.191890,4.668809,55.645210,48.265164,...,26.397863,16.768640,9.857595,23.965442,5.039568,4.011027,14.256306,14.064726,4.644481,5.960273
8458,MMP9,48.214583,68.906040,83.480858,59.044198,380.004006,284.778367,53.643662,606.568537,283.155629,...,268.595461,146.274134,140.281156,421.937018,78.640704,52.675420,165.747561,242.624830,91.341457,114.550358
9470,OSM,6.881496,14.702881,5.338475,8.263050,40.830175,19.915761,8.988251,49.782803,59.925555,...,31.647649,29.778156,48.340128,40.320040,11.895725,8.799701,25.020538,31.454446,5.540784,15.792548
9800,PFKFB3,39.656825,59.963616,27.516437,62.652745,249.411305,65.307810,63.616492,338.661277,436.098635,...,188.433827,148.374820,177.531490,233.205534,55.611049,52.593563,118.010535,104.005820,102.993401,78.745214
11818,SEMA6B,1.235140,0.877784,1.218176,0.993658,2.554380,1.418502,1.016339,2.931204,6.612475,...,4.058346,3.169457,3.222675,3.689226,0.644596,0.736719,2.664057,2.460496,1.140750,1.174652
12605,SOCS3,66.829911,131.941899,42.313688,83.362668,352.584329,138.474119,91.121177,331.774139,663.166306,...,285.387331,268.629922,583.967703,417.318224,105.655134,76.455078,240.881175,327.877683,44.896648,146.048443


In [39]:
#any column name that has the term HC is a Health Control
#any column with Base is a patient with CF but no Treatment
#any column with V2 is a CF patient with treatment

#### b. Split norm_cpm_df_top10 into healthy control and CF patients, two branches

In [68]:
# Split the data into hc_top10
hc_top10 = norm_cpm_df_1.loc[:,norm_cpm_df_1.columns.str.contains('HC')]

# Split the data into cf_top10
cf_top10 = norm_cpm_df_1.loc[:, norm_cpm_df_1.columns.str.contains('Base')]

In [70]:
gene_list = norm_cpm_df_1['gene_ids'].tolist()

In [71]:
gene_list

['ANXA3',
 'G0S2',
 'IL1R2',
 'LOC105372578',
 'MCEMP1',
 'MMP9',
 'OSM',
 'PFKFB3',
 'SEMA6B',
 'SOCS3']

#### c. Transpose data

In [72]:
hc_top10_t = hc_top10.T
hc_top10_t.columns=['ANXA3','G0S2','IL1R2','LOC105372578','MCEMP1','MMP9','OSM','PFKFB3','SEMA6B','SOCS3']
hc_top10_t.reset_index(level=0, inplace=True)
hc_top10_t.rename(columns = {'index':'Control group'}, inplace = True)

In [78]:
hc_top10_t['Y'] = 0

In [74]:
cf_top10_t = cf_top10.T
cf_top10_t.columns=['ANXA3','G0S2','IL1R2','LOC105372578','MCEMP1','MMP9','OSM','PFKFB3','SEMA6B','SOCS3']
cf_top10_t.reset_index(level=0, inplace=True)
cf_top10_t.rename(columns = {'index':'Treatment group'}, inplace = True)

In [79]:
cf_top10_t['Y'] = 1

In [80]:
hc_top10_t.head()

,Control group,ANXA3,G0S2,IL1R2,LOC105372578,MCEMP1,MMP9,OSM,PFKFB3,SEMA6B,SOCS3,Y
0,Norm_10_HC_Auto_066_237,55.272527,1.235140,55.228415,3.484860,5.866916,48.214583,6.881496,39.656825,1.235140,66.829911,0
1,Norm_13_HC_Auto_068_239,83.205775,1.464338,83.414966,4.759098,4.759098,59.044198,8.263050,62.652745,0.993658,83.362668,0
2,Norm_16_HC_Auto_072_243,81.402431,2.032679,83.847998,3.430145,4.668809,53.643662,8.988251,63.616492,1.016339,91.121177,0
3,Norm_19_HC_Auto_074_245,35.630254,1.991753,61.921398,2.832716,3.142544,22.971555,5.045775,41.516992,1.018007,42.490738,0
4,Norm_21_HC_Immune_004,88.458661,1.749091,88.264318,6.348553,5.668351,49.946267,9.911516,50.496907,1.068889,99.503848,0


In [81]:
cf_top10_t.head()

,Treatment group,ANXA3,G0S2,IL1R2,LOC105372578,MCEMP1,MMP9,OSM,PFKFB3,SEMA6B,SOCS3,Y
0,Norm_11_Orkambi_006_Base,71.045638,5.431288,93.483990,7.570887,4.882673,68.906040,14.702881,59.963616,0.877784,131.941899,1
1,Norm_14_Orkambi_007_Base,307.603285,7.224107,154.100984,29.854322,30.053883,380.004006,40.830175,249.411305,2.554380,352.584329,1
2,Norm_17_Orkambi_009_Base,545.561369,10.032819,183.188321,77.140705,55.645210,606.568537,49.782803,338.661277,2.931204,331.774139,1
3,Norm_1_Orkambi_001_Base,204.284840,9.517355,181.371018,33.152872,26.296768,233.513494,41.226837,250.022271,5.142078,491.113564,1
4,Norm_20_Orkambi_010_Base,425.051544,11.145413,139.251318,68.796387,36.952351,233.390250,42.326032,283.776804,4.544409,337.745811,1


#### d. Merge and generate hc_cf_top10

In [90]:
hc_cf_top10 = pd.concat([hc_top10_t, cf_top10_t],axis=0) 
cols = list(hc_cf_top10.columns)
cols = [cols[-1]] + cols[:-1]
hc_cf_top10 = hc_cf_top10[cols].reset_index()
hc_cf_top10 = hc_cf_top10.drop('index', axis=1)

### Question 4: SVM and cross validation

#### a. Split data into X and Y

In [105]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC

In [106]:
# Split hc_cf_top10 into X and y
X = hc_cf_top10.iloc[:,np.r_[2:12]]
y = hc_cf_top10.Y

In [107]:
# Perform SVM with cross validation = 5
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

clf = SVC(kernel='linear', C=1, random_state=1)
scores = cross_val_score(clf, X, y, cv=5)

In [108]:
scores

array([0.875, 0.75 , 0.875, 0.75 , 0.75 ])

In [ ]:

# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=0)

# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(
        SVC(), tuned_parameters, scoring='%s_macro' % score
    )
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()


In [ ]:
# different values for C ( less than 1, 1, and greater than 1)
# different kernels (linear and radial)

model = SVC(random_state = 1)
modelF = model.fit(X_train, y_train)
y_predF = modelF.predict(X_test)

In [116]:
# different values for C ( less than 1, 1, and greater than 1)
# different kernels (linear and radial)

model = SVC(random_state = 1)
modelF = model.fit(X_train, y_train)
y_predF = modelF.predict(X_test)

C = [0.01, 0.1, 1, 10, 100, 1000]
kernel = ['linear','rbf']
scores = ['precision', 'recall']

hyperF = dict(C = C, kernel = kernel)

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()
    
    hyperF = dict(C = C, kernel = kernel)

    gridF = GridSearchCV(model, hyperF, scoring='%s_macro' % score, cv = 5, verbose = 1, 
                      n_jobs = -1)

    bestF = gridF.fit(X_train, y_train)
    
    print("Best parameters set found on development set:")
    print()
    print(gridF.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = gridF.cv_results_['mean_test_score']
    stds = gridF.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, gridF.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, gridF.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

# Tuning hyper-parameters for precision

Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best parameters set found on development set:

{'C': 10, 'kernel': 'rbf'}

Grid scores on development set:

0.942 (+/-0.145) for {'C': 0.01, 'kernel': 'linear'}
0.290 (+/-0.040) for {'C': 0.01, 'kernel': 'rbf'}
0.942 (+/-0.145) for {'C': 0.1, 'kernel': 'linear'}
0.290 (+/-0.040) for {'C': 0.1, 'kernel': 'rbf'}
0.942 (+/-0.145) for {'C': 1, 'kernel': 'linear'}
0.835 (+/-0.544) for {'C': 1, 'kernel': 'rbf'}
0.942 (+/-0.145) for {'C': 10, 'kernel': 'linear'}
0.950 (+/-0.122) for {'C': 10, 'kernel': 'rbf'}
0.942 (+/-0.145) for {'C': 100, 'kernel': 'linear'}
0.917 (+/-0.139) for {'C': 100, 'kernel': 'rbf'}
0.942 (+/-0.145) for {'C': 1000, 'kernel': 'linear'}
0.917 (+/-0.139) for {'C': 1000, 'kernel': 'rbf'}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

              precision    recall  f1-score   sup

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  60 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Best parameters set found on development set:

{'C': 0.01, 'kernel': 'linear'}

Grid scores on development set:

0.917 (+/-0.211) for {'C': 0.01, 'kernel': 'linear'}
0.500 (+/-0.000) for {'C': 0.01, 'kernel': 'rbf'}
0.917 (+/-0.211) for {'C': 0.1, 'kernel': 'linear'}
0.500 (+/-0.000) for {'C': 0.1, 'kernel': 'rbf'}
0.917 (+/-0.211) for {'C': 1, 'kernel': 'linear'}
0.850 (+/-0.400) for {'C': 1, 'kernel': 'rbf'}
0.917 (+/-0.211) for {'C': 10, 'kernel': 'linear'}
0.900 (+/-0.245) for {'C': 10, 'kernel': 'rbf'}
0.917 (+/-0.211) for {'C': 100, 'kernel': 'linear'}
0.867 (+/-0.226) for {'C': 100, 'kernel': 'rbf'}
0.917 (+/-0.211) for {'C': 1000, 'kernel': 'linear'}
0.867 (+/-0.226) for {'C': 1000, 'kernel': 'rbf'}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

              precision    recall  f1-score   support

           0       0.56      0.83      0.67         6
           1       0.86      0.60    

[Parallel(n_jobs=-1)]: Done  45 out of  60 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    0.1s finished


#### b. Best combination of hyperparameters

In terms of precision, the best combination of hyperparameters is {'C': 10, 'kernel': 'rbf'}, where C parameter = 10 and kernel of radial basis function, the corresponding precision is 0.950.

In terms of recall, the best combination of hyperparameters is {'C': 0.01, 'kernel': 'linear'}, where C parameter = 0.01 and kernel of linear, the corresponding recall is 0.917.

#### c. Explain how the score that is reported is calculated.

The precision and recall scores are calculated by the actual testset values and predicted value of the model.

TP: true positive, number of observations are correctly classified as "Y = 1" when they actually with "Y = 1"

TN: true negative, number of observations are correctly classified as "Y = 0" when they actually with "Y = 0"

FP: false positive, number of observations are classified as "Y = 1" when they actually with "Y = 0"

FN: false negative, number of observations are classified as "Y = 0" when they actually with "Y = 1"

Precision = TP / (TP + FP)

Precision measure the rate that the model can correctly classified the actual CF patients into CF group based on the top 10 genes values.

Recall = TP / (TP + FN)

Recall measure the rate of the observations classified as CF group are actual CF patients based on the top 10 genes values.

In [117]:
hc_cf_top10.to_csv('/Users/likehang/Desktop/hc_cf_top10.xlsx')